In [ ]:
import math
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot
from random import shuffle
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.models import model_from_json
from keras.layers import Bidirectional
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import preprocessing
%matplotlib inline
from keras.layers import ConvLSTM2D

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (10, 10)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
File="/content/drive/My Drive/Tangni"

In [ ]:
data=pd.read_csv(File + '/weekly_angle.csv',header=0, usecols=[0, 1, 2, 3, 4, 5], index_col=0)

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(data)
df_normalized = pd.DataFrame(x_scaled)

In [ ]:
BH1=np.array(df_normalized[0])
BH2=np.array(df_normalized[1])
BH3=np.array(df_normalized[2])
BH4=np.array(df_normalized[3])
BH5=np.array(df_normalized[4])

In [ ]:
plt.plot(BH5)

In [ ]:
def rmse(predictions, targets):
    return np.sqrt(np.mean((predictions-targets)**2,axis=0))

In [ ]:
def fit_model(X, y, n_timesteps, n_features,node,epochs):

  model=Sequential()



  model.add(ConvLSTM2D(filters=node, kernel_size=(1, 2), activation='relu', input_shape=(None,1, n_timesteps,n_features)))
  model.add(Flatten())
  model.add(Dense(n_features))


  print(X.shape)
  print(X)
  model.compile(optimizer='adam',loss='mse')
  model.fit(X,y, epochs,verbose=0)

  return model

In [ ]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
  X, y = list(), list()
  for i in range(len(sequences)):
    # find the end of this pattern
    end_ix = i + n_steps_in
    out_end_ix = end_ix + n_steps_out
    # check if we are beyond the dataset
    if out_end_ix > len(sequences):
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequences[i:end_ix], sequences[end_ix:out_end_ix]
    X.append(seq_x)
    y.append(seq_y)
  return np.array(X), np.array(y)

In [ ]:
def time_series(BH1, BH2, BH3, BH4, BH5, n_steps, n_features):

  Train_BH1, Test_BH1=BH1[:62], BH1[62:]
  Train_BH2, Test_BH2=BH2[:62], BH2[62:]
  Train_BH3, Test_BH3=BH3[:62], BH3[62:]
  Train_BH4, Test_BH4=BH4[:62], BH4[62:]
  Train_BH5, Test_BH5=BH5[:62], BH5[62:]

  in_seq1 = Train_BH1.reshape((len(Train_BH1), 1))
  in_seq2 = Train_BH2.reshape((len(Train_BH2), 1))
  in_seq3 = Train_BH3.reshape((len(Train_BH3), 1))
  in_seq4 = Train_BH4.reshape((len(Train_BH4), 1))
  in_seq5 = Train_BH5.reshape((len(Train_BH5), 1))
  test_seq1 = Test_BH1.reshape((len(Test_BH1), 1))
  test_seq2 = Test_BH2.reshape((len(Test_BH2), 1))
  test_seq3 = Test_BH3.reshape((len(Test_BH3), 1))
  test_seq4 = Test_BH4.reshape((len(Test_BH4), 1))
  test_seq5 = Test_BH5.reshape((len(Test_BH5), 1))


  # horizontally stack columns
  dataset = np.hstack((in_seq1, in_seq2, in_seq3, in_seq4, in_seq5))
  test_dataset = np.hstack((test_seq1, test_seq2, test_seq3, test_seq4, test_seq5))
  # print(dataset)

  # convert into input/output
  X, y = split_sequences(dataset, n_steps,1)
  test_X, test_y = split_sequences(test_dataset, n_steps,1)

  

  return X, y, test_X, test_y

In [ ]:
def model(n_steps,node,epochs):

  n_features=5
  total_error=[]


  train_E=[]
  test_E=[] 



  
  X, y, test_X, test_y=time_series(BH1, BH2, BH3, BH4, BH5, n_steps, n_features, )
  X = X.reshape((X.shape[0], 1, 1, n_steps, n_features))
  test_X = test_X.reshape((test_X.shape[0], 1, 1, n_steps, n_features))
  model=fit_model(X, y, n_steps, n_features,node,epochs)
  yhat=model.predict(X,verbose=0)
  y=y.reshape(-1,5)
  test_y=test_y.reshape(-1,5)

  train_error=rmse(yhat, y)

  train_E.append(np.around(train_error,3))

  test_yhat=model.predict(test_X,verbose=0)
  test_error=rmse(test_yhat, test_y)
  test_E.append(np.around(test_error,3))

  Error1=[n_steps,node,epochs,list(train_E[0]),round(np.mean(train_E),3)]
  Error2=[n_steps,node,epochs,list(test_E[0]), round(np.mean(test_E), 3)]

  global max
  if Error2[-1]<max:
    max=Error2[-1]
    print('Minimum: ',Error2)
    global E1
    global E2 
    E1=Error1
    E2=Error2

  print(Error1)
  print(Error2)

  return y, yhat, test_y, test_yhat

  


In [ ]:
max=10
E1=[]
E2=[]
for n_steps in range(3,12):
  for node in range(16,256,16):
    for epochs in range(2000,2001,100):
      model(n_steps,node,epochs)



In [ ]:
print(E1)
print(E2)

In [ ]:
model(2,16,50)